In [1]:
import torch
import torch.nn.functional as F
from torch import nn

In [2]:
device = 'cuda:0'
from onsets_and_frames import dataset, summary

STFT filter created, time used = 0.0049 seconds
Mel filter created, time used = 0.0050 seconds


In [3]:
from evaluate import evaluate_wo_velocity

In [4]:
import matplotlib.pyplot as plt
import numpy as np

from tqdm import tqdm

onset_threshold=0.5
frame_threshold=0.5
save_path = './midi'

In [5]:
sequence_length = None
validation_length = sequence_length
# dataset = MAPS(groups=['AkPnBcht', 'AkPnBsdf', 'AkPnCGdD', 'AkPnStgb', 'SptkBGAm', 'SptkBGCl', 'StbgTGd2'], sequence_length=sequence_length)
validation_dataset = dataset.MAPS(groups=['ENSTDkAm', 'ENSTDkCl'], sequence_length=validation_length)

Loading group ENSTDkAm:   0%|          | 0/30 [00:00<?, ?it/s]

Loading 2 groups of MAPS at data/MAPS


Loading group ENSTDkCl: 100%|██████████| 30/30 [00:00<00:00, 161.54it/s]


In [6]:
model_file = './runs/BiTCNv2-600,500,400,300,200,100-200220-200915/model-2000.pt'

In [7]:
model = torch.load(model_file, map_location=device).eval()

In [8]:
summary(model)

OnsetsAndFrames_biTCNv2(
  (onset_stack): Onset_Stack_biv2(
    (layer1): ConvStack(
      (cnn): Sequential(
        (0): Conv2d(1, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), 480 params
        (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), 96 params
        (2): ReLU(), 0 params
        (3): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), 20,784 params
        (4): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), 96 params
        (5): ReLU(), 0 params
        (6): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False), 0 params
        (7): Dropout(p=0.25, inplace=False), 0 params
        (8): Conv2d(48, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), 41,568 params
        (9): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), 192 params
        (10): ReLU(), 0 params
        (11): MaxPool2d(kernel_size=(1, 2), st

47269760

In [9]:
with torch.no_grad():
    metrics = evaluate_wo_velocity(tqdm(validation_dataset), model, onset_threshold, frame_threshold, save_path)

100%|██████████| 60/60 [02:35<00:00,  1.62s/it]


In [12]:
for key, values in metrics.items():
    if key.startswith('metric/'):
        _, category, name = key.split('/')
        print(f'{category:>32} {name:25}: {np.mean(values):.3f} ± {np.std(values):.3f}')

                            note precision                : 0.819 ± 0.084
                            note recall                   : 0.672 ± 0.106
                            note f1                       : 0.737 ± 0.095
                            note overlap                  : 0.660 ± 0.080
               note-with-offsets precision                : 0.507 ± 0.135
               note-with-offsets recall                   : 0.419 ± 0.132
               note-with-offsets f1                       : 0.458 ± 0.133
               note-with-offsets overlap                  : 0.823 ± 0.078
                           frame f1                       : 0.642 ± 0.084
                           frame precision                : 0.804 ± 0.089
                           frame recall                   : 0.541 ± 0.099
                           frame accuracy                 : 0.478 ± 0.093
                           frame substitution_error       : 0.084 ± 0.040
                           frame miss_